In [ ]:
import time
import os
from elasticsearch import Elasticsearch
import requests
from piffle.iiif import IIIFImageClient
import json
import math
import numpy as np
import collections
from PIL import Image
from io import BytesIO

In [ ]:
def create_es_client():
    es = Elasticsearch(
        hosts=os.environ['LOCAL_HOST'],
        http_auth=(
            os.environ['LOCAL_USER'],
            os.environ['LOCAL_PASS']
        )
    )
    accepting_connections = es.ping()
    while not accepting_connections:
        time.sleep(1)
        accepting_connections = es.ping()        
    return es

def Extracting_url_ID(response):  # run through each page and get the url
    for doc in response['hits']['hits']:
        ID_url_dict = {'id': doc['_id'], 'url': doc['_source']['state']['derivedData']['thumbnail']['url']}
        yield ID_url_dict  # this is convenient!

def divide_api_request(image_url, k): #send request to clustering API
    header = {'image-url': image_url, 'k': k}
    response = requests.get('http://api:80/image/spacedivide/', headers = header)
    dr = json.loads(response.text)
    return dr

def get_pix_count(image_url):
    response = requests.get(image_url)
    imgpil = Image.open(BytesIO(response.content))
    pix_count = imgpil.size[0]*imgpil.size[1]
    return pix_count

def commondict2searchstr(most_common_dictionary, pix_count):
    searchstr = ''
    for key, value in most_common_dictionary.items():
        it_time = math.floor(value*100/pix_count)
        for i in range(it_time):
            searchstr = searchstr+key+' '
    return searchstr

In [ ]:
index_name = 'repetitive_search'

In [ ]:
def index_traversing(size, index_name):
    # run for 1 time and get a last_result
    es = create_es_client()
#     ####document this after first run
#     # create a new index
#     es.indices.create(
#     index=index_name,
#     )
#     ####
    response = es.search(
        index=os.environ['INDEX_NAME'],
        body={
            "query": {"match_all": {}},
            "size": size,
            "from": 0,
            "sort": {
                "_id": "asc",  # document ID.
            },
        }
    )
            
    k = 10
    
#     post the first 20 dictionaries
#     for ID_url_dict in Extracting_url_ID(response):
#         image_url = str(IIIFImageClient().init_from_url(ID_url_dict['url']).size(width=50))

#         pix_count = get_pix_count(image_url)
#         most_common_dictionary = divide_api_request(image_url, str(k))   #dividing in lab space.
#         most_common_list = list(most_common_dictionary)
#         searchstr = commondict2searchstr(most_common_dictionary, pix_count)
#         #update the document
#         es.index( 
#             index=index_name,
#             id=ID_url_dict['id'],
#             body={
#                 "iiif_url": ID_url_dict['url'],
#                 "image_url": image_url,
#                 "divide_info": {
#                     "most_common_dictionary": most_common_dictionary,
#                     "most_common_list": most_common_list,
#                     "searchstr": searchstr,
#                 }
#             }
#         )
#     last_result_id = response['hits']['hits'][size - 1]['_id']

    counter = size  # just to make the waiting time less painful
    
    
    ##############################
    
    last_result_id = 'a34zrjzb'
    print(last_result_id)
    ##############################
    
    
    
    while len(response['hits']['hits']) == size:  # get in the loop!
        response = es.search(
            index=os.environ['INDEX_NAME'],
            body={
                "query": {"match_all": {}},
                "size": size,
                "from": 0,
                "sort": {
                    "_id": "asc",  # document ID.
                },
                "search_after": [last_result_id],
            }
        )
        for ID_url_dict in Extracting_url_ID(response):

            image_url = str(IIIFImageClient().init_from_url(ID_url_dict['url']).size(width=50))
            pix_count = get_pix_count(image_url)
            most_common_dictionary = divide_api_request(image_url, str(k))   #dividing in lab space.
            most_common_list = list(most_common_dictionary)
            
            searchstr = commondict2searchstr(most_common_dictionary, pix_count)
            #update the document
            es.index( 
                index=index_name,
                id=ID_url_dict['id'],
                body={
                    "iiif_url": ID_url_dict['url'],
                    "image_url": image_url,
                    "divide_info": {
                        "most_common_dictionary": most_common_dictionary,
                        "most_common_list": most_common_list,
                        "searchstr": searchstr,
                    }
                }
            )
        last_result_id = ID_url_dict['id']  # update the last ID
        counter += size
        print(counter)
        print(last_result_id)


index_traversing(20, index_name)

In [ ]:
es = Elasticsearch(
    hosts=os.environ['LOCAL_HOST'],
    http_auth=(
        os.environ['LOCAL_USER'],
        os.environ['LOCAL_PASS']
    )
)

response = es.get(
    index=index_name,
    id='a37kz6j9'
)

response

In [ ]:
most_common_dictionary = response['_source']['divide_info']['most_common_dictionary']

In [ ]:
image_url = 'https://iiif.wellcomecollection.org/image/L0058653/full/50,/0/default.jpg'
response = requests.get(image_url)
imgpil = Image.open(BytesIO(response.content))
pix_count = imgpil.size[0]*imgpil.size[1]

In [ ]:
print(pix_count)